In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import random
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
from syn_DI_dataset import make_dataset, make_dataloader

### Loda Data

In [2]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
with open('config_3.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

['lidar']
S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16', 'A17', 'A18', 'A19', 'A2

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))

    # lengths = torch.tensor([t['input_mmwave'].shape[0] for t in batch ])

    # # rgb
    # rgb_data = np.array([(t['input_rgb']) for t in batch ])
    # rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # # depth
    # depth_data = np.array([(t['input_depth']) for t in batch ])
    # depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # # mmwave
    # ## padd
    # mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    # mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    # ## compute mask
    # mmwave_data = mmwave_data.permute(1,0,2)

    # lidar
    ## padd
    lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    ## compute mask
    lidar_data = lidar_data.permute(1,0,2)

    # # wifi-csi
    # wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    # wifi_data = torch.FloatTensor(wifi_data)
    "要改"
    exist_list = [False, False, False, True, False]

    # return rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, modality_list
    return lidar_data, kpts, exist_list

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

#### check random seed

In [17]:
print(len(train_loader.dataset)/16)

12991.5


In [5]:
avg_time = 0
for epoch in range(3):
    # random.seed(config['modality_existances']['train_random_seed'])
    i= 0
    for data in val_loader:
        # start_time = time.time()
        data, kpts, exist_list = data
        # epoch_time = time.time() - start_time
        # print(rgb_data[0].shape, depth_data[0].shape, lidar_data[0].shape, mmwave_data[0].shape, wifi_data[0].shape,kpts.shape, lengths.shape)
        print(data.shape, kpts.shape)
        # print('epoch_time: ', epoch_time)
        # avg_time += epoch_time
        # print(rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, modality_list)
        print(exist_list)
        i += 1
        if i > 5:
            print('............................................................................................')
            break
    # print('tot_time: ', avg_time)
    # print('avg_time: ', avg_time / (i+1))

C:\Users\CHEN_X~1\AppData\Local\Temp/ipykernel_51704/3752927639.py:39: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]


torch.Size([16, 1135, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1144, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1147, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1141, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1138, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1142, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
............................................................................................
torch.Size([16, 1135, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1144, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1147, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1141, 3]) torch.Size([16, 17, 3])
[False, False, False, True, False]
torch.Size([16, 1138, 3]) torch.Size([16, 17, 3])
[False,

### Model

In [6]:
'model without pos encoding'
from baseline_model import model

# model = model(['RGB'])
model = model(['Lidar'])
model.cuda()

# rgb_data = torch.randn(16, 3, 480, 640).cuda()
# depth_data = torch.randn(16, 3, 480, 640).cuda()
# lidar_data = torch.randn(16, 1432, 3).cuda()
# mmwave_data = torch.randn(16, 67, 5).cuda()
# wifi_data = torch.randn(16, 3, 114, 10).cuda()
# modality_list = [False, False, False, True, False]
# modality_list = [True, True, True, True, True]


# out = model(rgb_data, depth_data,  mmwave_data, lidar_data, wifi_data, modality_list)
# out = model(lidar_data, modality_list)
# print(out.shape)

model(
  (feature_extractor): single_feature_extrator(
    (lidar_extractor): lidar_feature_extractor(
      (fc1): Sequential(
        (0): Linear(in_features=3, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
      )
      (transformer1): TransformerBlock(
        (fc1): Linear(in_features=32, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=32, bias=True)
        (fc_delta): Sequential(
          (0): Linear(in_features=3, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=512, bias=True)
        )
        (fc_gamma): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=512, bias=True)
        )
        (w_qs): Linear(in_features=512, out_features=512, bias=False)
        (w_ks): Linear(in_features=512, out_features=512, bias=False)
     

### training 

In [7]:
def test(model, tensor_loader, criterion1, criterion2, device):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    # random.seed(config['modality_existances']['val_random_seed'])
    for data in tqdm(tensor_loader):
        # start_time = time.time()
        input_data, kpts, exist_list = data
        input_data = input_data.to(device)
        kpts.to(device)
        labels = kpts.type(torch.FloatTensor)
        outputs = model(input_data, exist_list)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        # t2 = time.time()
        # forward_time = t2 - t1
        test_mse += criterion1(outputs,labels).item() * input_data.size(0)

        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * input_data.size(0)
        test_pampjpe += pampjpe.item() * input_data.size(0)
        # t3 = time.time()
        # record_time = t3 - t2
        # print('load_time: ', load_time)
        # print('forward_time: ', forward_time)
        # print('record_time: ', record_time)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))
    return test_mpjpe

In [11]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
test(model, val_loader, train_criterion, test_criterion, device)

  1%|          | 39/3403 [00:12<18:28,  3.03it/s] 


KeyboardInterrupt: 

In [8]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, train_criterion, test_criterion, device):
    optimizer = torch.optim.AdamW(
        [
                {'params': model.regression_head.parameters()}
            ],
        lr = learning_rate
    )
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    parameter_dir = './baseline_weights/Lidar.pt'
    best_test_mpjpe = 100
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0
        random.seed(epoch)
        num_iter = 400
        for i, data in enumerate(tqdm(train_loader)):
            if i < num_iter:
                input_data, kpts, exist_list = data
                input_data = input_data.to(device)
                labels = kpts.to(device)
                labels = labels.type(torch.FloatTensor)
                
                optimizer.zero_grad()
                outputs = model(input_data, exist_list)
                # print(outputs)
                outputs = outputs.to(device)
                outputs = outputs.type(torch.FloatTensor)
                loss = train_criterion(outputs,labels)
                if loss == float('nan'):
                    print('nan')
                    print(outputs)
                    print(labels)
                    
                loss.backward()
                # print(length)
                # print("loss is ", loss.item())
                optimizer.step()
                
                epoch_loss += loss.item() * input_data.size(0)
            else:
                break
            # print("epoch loss is ", epoch_loss)
        # epoch_loss = epoch_loss/len(train_loader.dataset)
        epoch_loss = epoch_loss/(num_iter*input_data.size(0))
        print('Epoch: {}, Loss: {:.8f}'.format(epoch, epoch_loss))
        if (epoch+1) % 5 == 0:
            test_mpjpe = test(
                model=model,
                tensor_loader=test_loader,
                criterion1 = train_criterion,
                criterion2 = test_criterion,
                device= device
            )
            # if test_mpjpe <= best_test_mpjpe:
            #     print(f"best test mpjpe is:{test_mpjpe}")
            #     best_test_mpjpe = test_mpjpe
            #     torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    torch.save(model.state_dict(), parameter_dir)
    return

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

cuda:0


In [10]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./pre-train_weights/lidar_p1_random.pt'))
model.to(device)
train(
    model=model,
    train_loader= train_loader,
     test_loader= val_loader,    
    num_epochs= 35,
    learning_rate=1e-3,
    train_criterion = train_criterion,
    test_criterion = test_criterion,
    device=device 
        )

  3%|▎         | 400/12991 [09:25<4:56:32,  1.41s/it]


Epoch: 0, Loss: 0.09276865


  3%|▎         | 400/12991 [07:15<3:48:25,  1.09s/it]


Epoch: 1, Loss: 0.03203765


  3%|▎         | 400/12991 [06:26<3:22:55,  1.03it/s]


...

Epoch: 33, Loss: 0.01346437


  3%|▎         | 400/12991 [04:16<2:14:20,  1.56it/s]


Epoch: 34, Loss: 0.01317039


100%|██████████| 3403/3403 [08:12<00:00,  6.91it/s]

mse: 0.01240360, mpjpe: 0.16128206, pampjpe: 0.10352261
